In [1]:
import shutil
import os
import re
import time
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import PIL
import torch
import torchvision.transforms.functional as F
import torchvision.transforms as T
import torchvision
from torchvision.utils import flow_to_image
from torchvision.utils import save_image
from torchvision.models.optical_flow import raft_large
from torchvision.io import read_video
from pytube import YouTube
from tqdm.notebook import tqdm
import tempfile
from pathlib import Path
from urllib.request import urlretrieve
import ffmpeg
import winsound

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = raft_large(pretrained=True, progress=False).to(DEVICE)
model = model.eval()
plt.rcParams["savefig.bbox"] = "tight"
DEVICE

'cuda'

In [2]:
def plot(imgs, figsize=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]
    num_rows = len(imgs)
    num_cols = len(imgs[0])
    _, axs = plt.subplots(
        nrows=num_rows, ncols=num_cols, squeeze=False, figsize=figsize
    )
    for row_idx, row in enumerate(imgs):
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            img = F.to_pil_image(img.to("cpu"))
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    plt.tight_layout()


def preprocess(batch):
    # image dimension must be divisible by 8
    transforms = T.Compose(
        [
            T.ConvertImageDtype(torch.float32),
            T.Normalize(mean=0.5, std=0.5),  # map [0, 1] into [-1, 1]
            T.Resize(size=(HEIGHT, WIDTH)),
        ]
    )
    batch = transforms(batch)
    return batch


# def download_youtube(videourl, path):
#     yt = YouTube(videourl)
#     # title = yt.streams[0].title
#     yt = (
#         yt.streams.filter(progressive=True, file_extension="mp4")
#         .order_by("resolution")
#         .desc()
#         .first()
#     )
#     if not os.path.exists(path):
#         os.makedirs(path)
#     yt.download(path)
#     # return title


def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(data, key=alphanum_key)


def vid2frame(path_in):
    old_wd = os.getcwd()
    vidcap = cv2.VideoCapture(path_in)
    # success, image = vidcap.read()
    image = vidcap.read()[1]
    frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    path_out = path_in.replace(".mp4", "_frames/")
    os.makedirs(path_out, exist_ok=True)
    for o in os.listdir(path_out):
        os.remove(os.path.join(path_out, o))
    os.chdir(path_out)
    print("Converting " + path_in + " to frames:")
    for m in tqdm(range(frames)):
        cv2.imwrite("%d.jpg" % m, image)
        # success, image = vidcap.read()
        image = vidcap.read()[1]
    vidcap.release()
    os.chdir(old_wd)


def frame2vid(path_in, path_out, fps):
    old_wd = os.getcwd()
    # path_out = path_in.strip("/") + ".mp4"
    frame_array = []
    files = [f for f in os.listdir(path_in) if os.path.isfile(os.path.join(path_in, f))]
    files = sorted_alphanumeric(files)
    print("Reading " + path_in + " for conversion to video:")
    for m in tqdm(range(len(files))):
        filename = path_in + files[m]
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        frame_array.append(img)
    out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*"MP4V"), fps, size)
    print("Converting " + path_in + " to video:")
    for m in tqdm(range(len(frame_array))):
        out.write(frame_array[m])
    out.release()
    os.chdir(old_wd)


def concatenate_frames2video(
    path_in_of, path_video, path_out, start_time, end_time, fps
):
    old_wd = os.getcwd()
    frame_array = []
    files = [
        f for f in os.listdir(path_in_of) if os.path.isfile(os.path.join(path_in_of, f))
    ]
    files = sorted_alphanumeric(files)
    original_frames, _, _ = torchvision.io.read_video(
        path_video, start_pts=start_time, end_pts=end_time, pts_unit="sec"
    )
    original_frames = original_frames.permute(
        0, 3, 1, 2
    )  # (N, H, W, C) -> (N, C, H, W)
    num_frames = len(original_frames)
    indices = np.arange(0, num_frames)
    print("Reading " + path_in_of + " for conversion to video:")
    for m in tqdm(range(len(files))):
        filename = path_in_of + files[m]
        right_img = cv2.imread(filename)
        left_img = cv2.resize(
            np.array(F.to_pil_image(original_frames[m - 1]))[:, :, ::-1],
            (WIDTH, HEIGHT),
        )
        img = np.concatenate([left_img, right_img], axis=1)
        height, width, layers = img.shape
        size = (width, height)
        frame_array.append(img)
    out = cv2.VideoWriter(path_out, cv2.VideoWriter_fourcc(*"MP4V"), fps, size)
    print("Converting " + path_in_of + " to video:")
    for m in tqdm(range(len(frame_array))):
        out.write(frame_array[m])
    out.release()
    os.chdir(old_wd)


def cosine_distance_background_reduction(input_flow, threshold=1):
    input_flow_mean = torch.stack(
        (
            torch.ones(input_flow[0].size(), device=DEVICE) * input_flow[0].mean(),
            torch.ones(input_flow[0].size(), device=DEVICE) * input_flow[1].mean(),
        )
    )
    calculate_cosine_similarity = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    cosine_similarity = calculate_cosine_similarity(input_flow, input_flow_mean)
    mean_cosine_similarity = cosine_similarity.mean()
    indices_to_keep = cosine_similarity < mean_cosine_similarity * threshold
    output_flow = input_flow * indices_to_keep
    return output_flow

def get_grid(shape):
    x = torch.arange(1, shape[0] + 1, 1.0)
    y = torch.arange(1, shape[1] + 1, 1.0)

    return torch.meshgrid(x, y, indexing="ij")

# Operates on GPU
def fit_2d_surface(x_grid, y_grid, z):
    features = torch.stack([
        torch.ones_like(x_grid), x_grid, y_grid, x_grid * x_grid, x_grid * y_grid, y_grid * y_grid
    ]).reshape((6, -1)).T.cuda()

    solution = torch.linalg.lstsq(features, z.flatten())

    z_fitted = (features @ solution.solution).reshape(z.shape)

    z_residuals = z - z_fitted

    return solution, z_fitted, z_residuals

# Flow must be cuda of shape (2, H, W)
def get_detection_mask(flow, threshold):
    flow_magnitudes = torch.linalg.norm(flow, axis=0)
    x_grid, y_grid = get_grid(shape=flow_magnitudes.shape)

    solution, flow_magnitudes_fitted, flow_magnitude_residuals = fit_2d_surface(
        x_grid, y_grid, flow_magnitudes
    )

    return torch.abs(flow_magnitude_residuals) > threshold

def get_detection(flow, threshold=1.25):
    detection_mask = get_detection_mask(flow, threshold=threshold)

    return detection_mask * flow

def ffmpeg_xstack(input_path, output_path, *, fps):
    inputs = [

        ffmpeg.input(subdir / '%d.jpg', framerate=fps)
        for subdir in sorted(input_path.iterdir())
        if subdir.is_dir()
    ]

    ffmpeg \
        .filter(inputs, 'xstack', inputs=4, layout='0_0|w0_0|0_h0|w0_h0') \
        .output(str(output_path)) \
        .run(overwrite_output=True)

    return Video(output_path, width=1280, height=720)


def ffmpeg_hstack(video_path, frames_path, output_path, *, fps):
    inputs = [
        ffmpeg.input(video_path, ss=FROM, to=TO),
        ffmpeg.input(frames_path / '%d.jpg', framerate=fps)
    ]

    ffmpeg \
        .filter(inputs, 'hstack') \
        .output(str(output_path)) \
        .run(overwrite_output=True)

    return Video(output_path, width=1280, height=720)

In [ ]:
thresholds = np.array([1])

THRESHOLD = 1
HEIGHT = 720
WIDTH = 1280
ALGORITHM = "cosine"
INPUT_PATH = "C:/Users/pv5/Desktop/Clock_Face_3Videvo_preview.mp4"
OUTPUT_PHOTOS_PATH = "C:/pv5/pv5/Desktop/bike1_flow/"
OUTPUT_VIDEO_PATH = INPUT_PATH.strip(".mp4") + "_flow"
os.makedirs(OUTPUT_PHOTOS_PATH, exist_ok=True)

frames, list_of_flows, predicted_flows, flow_imgs, grid = [], [], [], [], []

START_TIME = 0
END_TIME = 7 #add 1 extra second

frames, _, _ = read_video(
    INPUT_PATH, start_pts=START_TIME, end_pts=END_TIME, pts_unit="sec"
)
frames = frames.permute(0, 3, 1, 2)  # (N, H, W, C) -> (N, C, H, W)

num_frames = len(frames)
FPS = num_frames / (END_TIME - START_TIME)
indices = np.arange(0, num_frames)
reduction_times = np.array([])
full_flows_stack = torch.empty([(len(indices) - 2), 2, HEIGHT, WIDTH])
torch.cuda.empty_cache()

for THRESHOLD in thresholds:
    print("Calculating optical flow:")
    for i in tqdm(indices[0:-2]):
            img_1 = frames[i : (i + 1)]
            img_2 = frames[(i + 1) : (i + 2)]
            img_1_preprocessed = preprocess(img_1).to(DEVICE)
            img_2_preprocessed = preprocess(img_2).to(DEVICE)
            with torch.no_grad():
                list_of_flows = model(
                    img_1_preprocessed.to(DEVICE), img_2_preprocessed.to(DEVICE)
                )
            flow = list_of_flows[-1][0]
            full_flows_stack[i] = flow
            t_0 = time.time()
            if ALGORITHM == "cosine":
                flow = cosine_distance_background_reduction(flow, threshold=float(THRESHOLD))
            elif ALGORITHM == "plane":
                flow = get_detection(flow, threshold=float(THRESHOLD))
            elif ALGORITHM != "none":
                break
            t_1 = time.time()
            reduction_times = np.append(reduction_times, t_1 - t_0)
            flow_img = flow_to_image(flow)
            (F.to_pil_image(flow_img)).save(OUTPUT_PHOTOS_PATH + str(i) + ".jpg")
        
    
    OUTPUT_VIDEO_PATH = (
        OUTPUT_VIDEO_PATH
        + "_"
        +str(START_TIME)
        +"-"
        +str(END_TIME)
        +"-"
        + ALGORITHM
        + "_"
        + str(THRESHOLD)
        + "_"
        + str(np.mean(reduction_times)).replace(".", ",")
        + ".mp4"
    )
    torch.save(full_flows_stack, OUTPUT_VIDEO_PATH.replace(".mp4",".pt"))
    concatenate_frames2video(
        OUTPUT_PHOTOS_PATH, INPUT_PATH, OUTPUT_VIDEO_PATH, START_TIME, END_TIME, FPS
    )

    frame2vid(OUTPUT_PHOTOS_PATH, OUTPUT_VIDEO_PATH, FPS)
    

duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)